In [1]:
# import kenlm
import numpy as np
import re
import os
import string
import util
import itertools
from helpers import *
import sys
from decimal import Decimal

In [2]:
# Open the file
with open('count_unigrams.txt','r') as f:
    unigram_counts = f.readlines()

with open('count_bigrams.txt', 'r') as fb:
    bigram_counts = fb.readlines()

In [3]:
'''
This function takes in a word and a count and splits it into the word and the integer.
ex) parse_word_count('the\t23135851162\n') = (the, 23135851162)
'''
def parse_unigram_count(string):
    return (string.split("\t")[0], int(string.split("\t")[1].split("\n")[0]))

'''
This function takes in 2 words and a count and splits it into the words and the integer.
ex) parse_bigram_count ('important area\t201828\n') = (important, area, 201828)
'''
def parse_bigram_count(string):
    return (string.split("\t")[0].split(" ")[0], string.split("\t")[0].split(" ")[1],int(string.split("\t")[1].split("\n")[0]))

In [4]:
# A dictionary with (word: probability) pairs for 1/3 million most frequent English words 
unigram_dict = {parse_unigram_count(line)[0]:parse_unigram_count(line)[1] for line in unigram_counts}
# A dictionary with (word + word: probability) apirs for 1/4 million most frequent English bigrams
bigram_dict = {(parse_bigram_count(line)[0] + " " + parse_bigram_count(line)[1]):parse_bigram_count(line)[2] for line in bigram_counts}

# Unigrams

First, let's consider a unigram model, where the probability of seeing each word is independent of the words before and after it.
P(W1:n)=Πk=1:nP(Wk)

Now we need to create a probability distribution dictionary unigram_probs from unigram_dict. But we will examine many, many "words" that are not truly words. Our unigram_probs dictionary must return a very small value in that case. (We do not want to simply return 0, because Jane Austen may use her own words (made up words, proper nouns, etc.) that are not in our data set. Instead, following Peter Norvig (CITE), we'll create a class that returns a probability given a string.

In [20]:
class ngram_Prob(dict):
    def __init__(self, data, N, fn):
        for key,count in data.iteritems():
            self[key] = count
        self.N = float(N)
        self.fn = fn
    def __call__(self, key):
        if key in self:
            return self[key]/self.N
        else: 
            return self.fn(key, self.N)


'''
This function returns a probailtiy on unrecognized words. 
It makes it more unlikely for long unrecognized words to be used than short unrecognized words.
The probability is inversely proportional to the length of the unrecognized word.
'''
def avoid_long_words(word, N):
    return Decimal(10.)/(Decimal(N) * Decimal(10)**Decimal(len(word)))

N = 1024908267229 ## Number of tokens in corpus
p_unigram = ngram_Prob(unigram_dict, N, avoid_long_words)

In [21]:
'''
This memoizing function caches the results of previous calls to the segment
function so that each results doesn't have to be recomputed.

ex) segment("helloworld") doesn't have to compute segment("lloworld") except for once.

The memoizing function helps segment only call itself n times, rather than 2^n times.
'''
def memoize(function):
    memo = {}
    def helper(x):
        if x not in memo.keys():
            memo[x] = function(x)
        return memo[x]
    return helper

In [22]:
'''
Parameters
    Corpus: The flattened text which we need to segment.
Output
    A list of words, separated according to our probability model.

Ex) segment('helloworld') = ['hello','world']
'''

def tupleToString(tup):
    string = ''
    for i in range(len(tup)):
        if i == len(tup)-1:
            string += tup[i]
        else:
            string += (tup[i]+' ')
    return string

@memoize
def segment(corpus):
    if not corpus: 
        return []
    candidates = tuple([first]+segment(remaining) for first,remaining in splits(corpus))
    return max(candidates, key=Pwords)

def splits(text, L=20):
    '''
    Return a list of all possible splits of the text, where length(first word)<=L.
    '''
#     return [(text[:i+1], text[i+1:]) for i in range(min(len(text), L))]
#     return [(text[:i+1], text[i+1:]) for i in range(min(len(text), L))]
    return [(text[:i+1], text[i+1:]) for i in range(len(text))]

def Pwords(words):
    '''
    The Naive Bayes probability of a sequence of words.
    '''
    return product([Decimal(uProb(w)) for w in words])

def uProb(word):
    '''
    Returns the unigram probability of a word by consulting unigram data.
    '''
    return p_unigram.__call__(word)

In [23]:
# Let's define new functions that use log probability to avoid overflow.
from math import log10
def log_prob(words):
    return sum([log10(uProb(w)) for w in words])

In [47]:
doc = alphaLowerNoSpace("senseAndSensibilityLast.txt")

### Brett-This is my iterative version of the recursive function for unigrams only right now. It's working, and it's reasonably fast, but I think there might be a way to speed it up.

In [44]:
# Returns a dictionary of segments.
def segmentString(string):
    docs = splits(string)
    docs.reverse()
    docs.append((' ', string))
    segments = {}
    for first, remaining in docs:
        if remaining == '':
            segments[remaining] = [], Decimal(1.)
        else:
            rem = splits(remaining)
            rem.reverse()
#             segments[remaining] = max(([f] + segments[r] for f,r in rem), key=Pwords)
            currentScore, bestScore, bestList = Decimal(0.), Decimal(0.), []
            for f,r in rem:
                currentList = [f] + segments[r][0]
                currentScore = Pwords([f]) * segments[r][1]
                if currentScore > bestScore: 
                    bestScore = currentScore
                    bestList = currentList
            segments[remaining] = bestList, bestScore
    return segments[string]

In [46]:
%time
segmentString(doc[:800])
# segmentString('wheninthecourseofhumanevents')

(['after',
  'a',
  'proper',
  'resistance',
  'on',
  'the',
  'part',
  'of',
  'mrs',
  'ferrars',
  'just',
  'so',
  'violent',
  'and',
  'so',
  'steady',
  'as',
  'to',
  'preserve',
  'her',
  'from',
  'that',
  'reproach',
  'which',
  'she',
  'always',
  'seemed',
  'fearful',
  'of',
  'incurring',
  'the',
  'reproach',
  'of',
  'being',
  'too',
  'amiable',
  'edward',
  'was',
  'admitted',
  'to',
  'her',
  'presence',
  'and',
  'pronounced',
  'to',
  'be',
  'again',
  'her',
  'son',
  'her',
  'family',
  'had',
  'of',
  'late',
  'been',
  'exceedingly',
  'fluctuating',
  'for',
  'many',
  'years',
  'of',
  'her',
  'life',
  'she',
  'had',
  'had',
  'two',
  'sons',
  'but',
  'the',
  'crime',
  'and',
  'annihilation',
  'of',
  'edward',
  'a',
  'few',
  'weeks',
  'ago',
  'had',
  'robbed',
  'her',
  'of',
  'one',
  'the',
  'similar',
  'annihilation',
  'of',
  'robert',
  'had',
  'left',
  'her',
  'for',
  'a',
  'fortnight',
  'without'

# Bigrams

Let's make a probability dictionary out of our bigram_counts dictionary called `p_bigram`.

In [46]:
# 1024908267229 is the number of tokens in the corpus
p_bigram = ngram_Prob(bigram_dict, 1024908267229, avoid_long_words)

Now let's make a function `cond_prob` which returns P(word | previous_word).

In [47]:
def cond_prob(word, prev):
    "The conditional probability P(word | previous-word)." 
    try:
        return p_bigram[prev + ' ' + word]/float(p_unigram[prev])
    # If the bigram is not documented in the top 1/4 million English bigrams, just return unigram(word).
    except KeyError:
        return p_unigram(word)

In [79]:
from math import log10
def memoize2(function):
    memo = {}
    def helper(x,y):
        if (x,y) not in memo.keys():
            memo[(x,y)] = function(x,y)
        return memo[(x,y)]
    return helper

@memoize2
def segment2(text, prev='<S>'):
    "Return (log P(words), words), where words is the best segmentation."
    if not text: return 0.0, []
    candidates = [combine(log10(cond_prob(first, prev)), first, segment2(rem, first)) for first,rem in splits(text)]
    return max(candidates)
                                                            
def combine(Pfirst, first, (Prem, rem)):
    "Combine first and rem results into one (probability, words) pair."
    # We add here because we're dealing with the logs
    return Pfirst+Prem, [first]+rem    

In [80]:
segment2("wheninthecourseofhumaneventsitbecomesnecessary", '<S>')

(-24.959286263365577,
 ['when',
  'in',
  'the',
  'course',
  'of',
  'human',
  'events',
  'it',
  'becomes',
  'necessary'])